In [2]:
import pandas as pd
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sqlalchemy import create_engine

# 1. Conexión a la Base de Datos SQL Server
def establecer_conexion():
    connection = pyodbc.connect(
        Trusted_Connection='Yes',
        Driver="{SQL Server Native Client 11.0}",
        Server='NB200HNN\SQLEXPRESS',
        Database='College_DT'
    )
    return connection

# 2. Recopilación de Datos
def obtener_datos(conexion, consulta_sql):
    datos = pd.read_sql(consulta_sql, conexion)
    return datos

# 3. Limpieza y Preparación de Datos (preprocesamiento aquí si es necesario)

# Codificar la columna 'Nivel'
def codificar_nivel(datos):
    datos['Nivel'] = datos['Nivel'].astype('category')
    datos['Nivel'] = datos['Nivel'].cat.codes
    return datos


# 4. Análisis de Datos y Cálculos
def calcular_probabilidad_paso_5to_grado(datos):
    # Aquí, calculamos la probabilidad de paso de 5to grado en función de las notas.
    
    # Supongamos que tienes una columna llamada "Notas_5to_grado" que representa las calificaciones
    # de los alumnos en 5to grado. Puedes usar esto para calcular la probabilidad.
    
    # Calculamos la probabilidad como el porcentaje de alumnos con calificaciones
    # iguales o superiores a 12 o que tienen la nota "C":
    alumnos_pasantes = datos[(datos['PasoAnio'] == '1') & (datos['Grado'] == '5')]
    probabilidad1 = len(alumnos_pasantes) / len(datos['AlumnoID'])
    print(probabilidad1)
    return probabilidad1

def calcular_probabilidad_paso_global(datos):
    # Aquí, calculamos la probabilidad de paso global en función de las notas.
    
    # Calculamos la probabilidad como el porcentaje de alumnos con un promedio general
    # igual o superior a 12 o que tienen la nota "C" en algún grado:
    alumnos_pasantes = datos[(datos['PasoAnio'] == '1')]
    probabilidad2 = len(alumnos_pasantes) / len(datos['AlumnoID'])
    print(probabilidad2)
    return probabilidad2

# 5. Implementación de Regresión Logística
def implementar_regresion_logistica(datos, etiquetas):
    # Divide los datos en conjunto de entrenamiento y prueba
    x_train, x_test, y_train, y_test = train_test_split(datos, etiquetas, test_size=0.3, random_state=1)

    # Crea y ajusta el modelo de regresión logística
    model = LogisticRegression()
    model.fit(x_train, y_train)

    # Predicciones y métricas
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(y_pred)
    return model, accuracy, report

# 6. Predicciones y Resultados
def realizar_predicciones(model, datos):
    # Realiza predicciones utilizando el modelo entrenado
    predicciones = model.predict(datos)
    return predicciones

# 7. Inserción de Resultados en la Base de Datos
def insertar_resultados(conexion, resultados):
    # Crea una tabla en la base de datos para almacenar los resultados
    resultados.to_sql('Resultados', conexion, if_exists='replace', index=False)

# 8. Ejecución del Algoritmo
if __name__ == "__main__":
    conexion = establecer_conexion()

    # 2. Recopilación de Datos
    consulta_sql = "SELECT * FROM Alumnado_2020"
    datos_alumnos = obtener_datos(conexion, consulta_sql)

    # Función para asignar valores numéricos a las letras
    def asignar_valor_numerico(letra):
        if letra == 'AD':
            return 5
        elif letra == 'A':
            return 4
        elif letra == 'B':
            return 3
        elif letra == 'C':
            return 2
        elif letra == 'D':
            return 1
        else:
            return 0  # Valor predeterminado si hay valores no válidos

    # Aplicar la función de asignar_valor_numerico a las columnas
    datos_alumnos['Promedio General'] = datos_alumnos['Promedio General'].apply(asignar_valor_numerico)
    datos_alumnos['Promedio de Asistencias'] = datos_alumnos['Promedio de Asistencias'].apply(asignar_valor_numerico)

    # 3. Limpieza y Preparación de Datos
    datos_alumnos = codificar_nivel(datos_alumnos)
    datos_alumnos = datos_alumnos.drop(["Año de Registro", "NombreCompleto"], axis=1)

    # 4. Análisis de Datos y Cálculos
    probabilidades_paso_5to = calcular_probabilidad_paso_5to_grado(datos_alumnos)
    probabilidades_paso_global = calcular_probabilidad_paso_global(datos_alumnos)

    # 5. Implementación de Regresión Logística
    etiquetas = datos_alumnos["PasoAnio"]
    datos_entrenamiento = datos_alumnos.drop(["PasoAnio"], axis=1)
    modelo_logistico, accuracy, reporte = implementar_regresion_logistica(datos_entrenamiento, etiquetas)

    # Asegúrate de eliminar la columna 'PasoAnio' de 'datos_alumnos' antes de hacer predicciones.
    datos_alumnos_sin_PasoAnio = datos_alumnos.drop(['PasoAnio'], axis=1)

    # 6. Predicciones y Resultados
    # Realiza predicciones utilizando el modelo entrenado y el conjunto de datos sin 'PasoAnio'.
    predicciones = realizar_predicciones(modelo_logistico, datos_alumnos_sin_PasoAnio)

    # 7. Inserción de Resultados en la Base de Datos
    resultados_df = pd.DataFrame({
        "AlumnoID": datos_alumnos["AlumnoID"],
        "Grado": datos_alumnos["Grado"],
        "Nivel": datos_alumnos["Nivel"],
        "PrediccionesRegLogistica": predicciones
    })

    # Crea una cadena de conexión compatible con SQLAlchemy
    conn_str = (
        "DRIVER={SQL Server Native Client 11.0};"
        "SERVER=NB200HNN\SQLEXPRESS;"
        "DATABASE=College_DT;"
        "Trusted_Connection=yes;"
    )
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

    # 8. Inserta los resultados en la base de datos
    insertar_resultados(engine, resultados_df)

    # Cierra la conexión
    conexion.close()


0.24
0.81
['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '0' '1' '1' '1' '0' '1' '1' '1' '0' '1' '1' '1']


C:\Users\Administrador\AppData\Local\Temp\ipykernel_20224\1545016941.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datos = pd.read_sql(consulta_sql, conexion)
